In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/ngtnkiet/Downloads/hello_ds/dataset/cleaned_2018.csv')
df.head()

,date,article_title,stock_symbol
0,2018-11-26 00:00:00+00:00,Asian ADRs Move Higher in Monday Trading,AACG
1,2018-11-14 00:00:00+00:00,Asian ADRs Move Higher in Wednesday Trading,AACG
2,2018-11-09 00:00:00+00:00,Asian ADRs Move Lower in Friday Trading,AACG
3,2018-10-26 00:00:00+00:00,Asian ADRs Move Lower in Friday Trading,AACG
4,2018-10-25 00:00:00+00:00,Asian ADRs Move Higher in Thursday Trading,AACG


In [3]:
# Assuming df is your DataFrame and 'date_column' is the column with dates
df['date'] = pd.to_datetime(df['date'])
df = df[(df['date'] >= '2018-01-01') & (df['date'] <= '2018-12-31')]
df.reset_index(drop=True, inplace=True)

In [4]:
df.head()

,date,article_title,stock_symbol
0,2018-11-26 00:00:00+00:00,Asian ADRs Move Higher in Monday Trading,AACG
1,2018-11-14 00:00:00+00:00,Asian ADRs Move Higher in Wednesday Trading,AACG
2,2018-11-09 00:00:00+00:00,Asian ADRs Move Lower in Friday Trading,AACG
3,2018-10-26 00:00:00+00:00,Asian ADRs Move Lower in Friday Trading,AACG
4,2018-10-25 00:00:00+00:00,Asian ADRs Move Higher in Thursday Trading,AACG


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654629 entries, 0 to 654628
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   date           654629 non-null  datetime64[ns, UTC]
 1   article_title  654629 non-null  object             
 2   stock_symbol   654629 non-null  object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 15.0+ MB


In [6]:
!pip install textblob


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [7]:
from textblob import TextBlob

In [8]:
# Assuming df is your DataFrame and 'article_title' is the column with text data
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

In [9]:
# Apply the sentiment analysis function to the 'article_title' column
df['textblob'] = df['article_title'].apply(get_sentiment)

In [10]:
df.head()

,date,article_title,stock_symbol,textblob
0,2018-11-26 00:00:00+00:00,Asian ADRs Move Higher in Monday Trading,AACG,0.125
1,2018-11-14 00:00:00+00:00,Asian ADRs Move Higher in Wednesday Trading,AACG,0.125
2,2018-11-09 00:00:00+00:00,Asian ADRs Move Lower in Friday Trading,AACG,0.000
3,2018-10-26 00:00:00+00:00,Asian ADRs Move Lower in Friday Trading,AACG,0.000
4,2018-10-25 00:00:00+00:00,Asian ADRs Move Higher in Thursday Trading,AACG,0.125


In [11]:
# Load the S&P 500 stock symbols (Ensure this file contains a column 'Symbol' with ticker symbols)
sp500_tickers = pd.read_csv("/Users/ngtnkiet/Downloads/hello_ds/dataset/constituents-2.csv")  # Replace with actual file path
sp500_list = sp500_tickers['Symbol']
print('Total number of tickers:', sp500_list.value_counts().sum())


Total number of tickers: 503


In [12]:
sp500_list = sp500_list.tolist()

df['date'] = pd.to_datetime(df['date']).dt.date
daily_aggregated_df = df.groupby(['date', 'stock_symbol'])['textblob'].mean().reset_index()

pivot_df = daily_aggregated_df.pivot(index='date', columns='stock_symbol', values='textblob').reset_index()

# Filter the pivot DataFrame to keep only S&P 500 tickers
filtered_pivot_df = pivot_df[['date'] + [col for col in pivot_df.columns if col in sp500_list]]


In [13]:
filtered_pivot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Columns: 439 entries, date to ZTS
dtypes: float64(438), object(1)
memory usage: 1.2+ MB


In [14]:
nasdaq_df = pd.read_csv("/Users/ngtnkiet/Downloads/hello_ds/dataset/nasdaq_historical_data.csv")

# Convert all column names in nasdaq_df to lowercase
nasdaq_df.columns = nasdaq_df.columns.str.lower()

nasdaq_df.head()

,date,close/last,open,high,low
0,2/11/25,6068.50,6049.32,6076.28,6042.34
1,2/10/25,6066.44,6046.40,6073.38,6044.84
2,2/7/25,6025.99,6083.13,6101.28,6019.96
3,2/6/25,6083.57,6072.22,6084.03,6046.83
4,2/5/25,6061.48,6020.45,6062.86,6007.06


In [15]:
# Ensure the 'date' column is in datetime format and then convert to date only
nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date']).dt.date

# Filter the DataFrame to only include rows where the date is from January to March 2018
nasdaq_df = nasdaq_df[(nasdaq_df['date'] >= pd.to_datetime('2018-01-01').date()) & (nasdaq_df['date'] <= pd.to_datetime('2018-12-31').date())]

# Sort the filtered DataFrame by date
nasdaq_df = nasdaq_df.sort_values(by='date')

# Reset the index of the DataFrame
nasdaq_df.reset_index(drop=True, inplace=True)

# Display the first few rows of the sorted and filtered DataFrame
nasdaq_df.head()

/var/folders/v9/qbwgkp_97lj_qtmnz0f8hj100000gn/T/ipykernel_9447/197032827.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date']).dt.date


,date,close/last,open,high,low
0,2018-01-02,2695.81,2683.73,2695.89,2682.36
1,2018-01-03,2713.06,2697.85,2714.37,2697.77
2,2018-01-04,2723.99,2719.31,2729.29,2719.07
3,2018-01-05,2743.15,2731.33,2743.45,2727.92
4,2018-01-08,2747.71,2742.67,2748.51,2737.60


In [16]:
# Create a new column "Price Change" as the difference between "Close/Last" and "Open" and round to 2 decimal places
nasdaq_df['price_change'] = ((nasdaq_df['close/last'] - nasdaq_df['open']).round(2))/nasdaq_df['open']

# Keep only the "Date" and "Price Change" columns
nasdaq_df = nasdaq_df[['date', 'price_change']]

In [17]:
# Perform a left join on nasdaq_df with filtered_pivot_df on the 'date' column
merged_df = pd.merge(nasdaq_df, filtered_pivot_df, on='date', how='outer')

# Display the first few rows of the merged DataFrame
merged_df.head()

,date,price_change,A,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
0,2018-01-02,0.004501,NaN,0.500000,0.25,-0.09375,NaN,0.5,0.142857,NaN,...,0.177778,0.0,NaN,0.20,NaN,NaN,NaN,0.0,NaN,0.0
1,2018-01-03,0.005638,NaN,0.000000,0.50,0.00000,0.000000,0.0,0.171429,NaN,...,0.130556,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2018-01-04,0.001721,0.0,0.040341,0.00,0.25000,0.107143,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-05,0.004328,NaN,0.000000,0.00,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,0.35,NaN,NaN,NaN,0.0,NaN,NaN
4,2018-01-08,0.001838,NaN,0.000000,NaN,NaN,NaN,NaN,0.057407,NaN,...,0.000000,NaN,NaN,0.00,NaN,0.7,0.178788,0.0,NaN,0.0


In [18]:
merged_df.to_csv('merged_data.csv', index=False)